In [1]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
import  pyspark.sql.functions as F

In [2]:
config = configparser.ConfigParser()
config.read('dl.cfg')

['dl.cfg']

In [3]:
os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']

In [4]:
def create_spark_session():
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark

In [5]:
spark = create_spark_session()

In [6]:
spark

In [29]:
import boto3
bucket = 'udacity-dend'
#Make sure you provide / in the end
prefix = 'song_data/A/'  
file_locations = []

client = boto3.client('s3')
result = client.list_objects(Bucket=bucket, Prefix=prefix, Delimiter='/')
for o in result.get('CommonPrefixes'):
    file_locations.append(o.get('Prefix'))

print("# of file_locations:", len( file_locations))
file_locations = ["s3a://udacity-dend/" + s for s in file_locations]
file_locations = [ s + "*" for s in file_locations]

# of file_locations: 26


In [50]:
import boto3
bucket = 'udacity-dend'
#Make sure you provide / in the end
prefix = 'song_data/A/'  
file_locations = []

client = boto3.client('s3')
result = client.list_objects(Bucket=bucket, Prefix=prefix, Delimiter='/')
for o in result.get('CommonPrefixes'):
    file_locations.append(o.get('Prefix'))

print("# of file_locations:", len( file_locations))
file_locations = ["s3a://udacity-dend/" + s for s in file_locations]
file_locations = [ s + "*" for s in file_locations]

# of file_locations: 26


In [52]:
s3path = 'https://s3.console.aws.amazon.com/s3/buckets/udacity-dend/?region=us-west-2#'
s3pathsong = 'https://s3.console.aws.amazon.com/s3/buckets/udacity-dend/song_data/A/?region=us-west-2'


In [53]:
import boto3

s3 = boto3.resource('s3')
udacity_bucket = s3.Bucket('udacity-dend')
songs_paths = []

for object_summary in udacity_bucket.objects.filter(Prefix=prefix):
    songs_paths.append(object_summary.key)
    print(object_summary.key)

song_data/A/Z/D/TRAZDFN12903CE9596.json
song_data/A/Z/D/TRAZDFY128F4217B71.json
song_data/A/Z/D/TRAZDFZ128F426FC56.json
song_data/A/Z/D/TRAZDGW128F424A4A3.json
song_data/A/Z/D/TRAZDHF128F147652B.json
song_data/A/Z/D/TRAZDHN12903CB7532.json
song_data/A/Z/D/TRAZDID128F92DFE93.json
song_data/A/Z/D/TRAZDJA128F42282CC.json
song_data/A/Z/D/TRAZDJL128F147659D.json
song_data/A/Z/D/TRAZDMY128E078B2D6.json
song_data/A/Z/D/TRAZDNB128F146B048.json
song_data/A/Z/D/TRAZDPF12903CF129D.json
song_data/A/Z/D/TRAZDPO128E078ECE6.json
song_data/A/Z/D/TRAZDRF12903CE41F0.json
song_data/A/Z/D/TRAZDTL128F92D4A05.json
song_data/A/Z/D/TRAZDVH12903CDD5AC.json
song_data/A/Z/D/TRAZDYQ12903CF43F6.json
song_data/A/Z/D/TRAZDZB128F428CF4A.json
song_data/A/Z/D/TRAZDZH128E0787878.json
song_data/A/Z/E/TRAZEBG128F1456D2E.json
song_data/A/Z/E/TRAZECG128F426471C.json
song_data/A/Z/E/TRAZEDI128E0784866.json
song_data/A/Z/E/TRAZEDX128F1488A82.json
song_data/A/Z/E/TRAZEET128F92D98B2.json
song_data/A/Z/E/TRAZEGV128F1499181.json


In [48]:
for key in bucket.list(prefix=prefix): 
    print(key)

AttributeError: 'str' object has no attribute 'list'

In [51]:
for my_bucket_object in bucket.objects.all():
    print(my_bucket_object)

AttributeError: 'str' object has no attribute 'objects'

In [44]:
bucket.list_objects()

AttributeError: 'str' object has no attribute 'list_objects'

In [8]:
#file_locations

In [8]:
# Not sure that second line is faster than first
#df = spark.read.format("json").load("s3a://udacity-dend/song_data/*/*/*")
df = spark.read.format("json").load(file_locations)

In [9]:
print(type(df))

<class 'pyspark.sql.dataframe.DataFrame'>


In [ ]:
"s3://udacity-dend/song_data/"

In [10]:
print(type(df))
print(df)
print('count', df.count())
#df.printSchema()

<class 'pyspark.sql.dataframe.DataFrame'>
DataFrame[artist_id: string, artist_latitude: double, artist_location: string, artist_longitude: double, artist_name: string, duration: double, num_songs: bigint, song_id: string, title: string, year: bigint]
count 14896
root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



In [11]:
df.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



In [11]:
#df.show(5)

In [ ]:
os.path.join("s3a://udacity-dend/", "song_data")

In [ ]:
'''
import boto3 
s3 = boto3.client("s3")
all_objects = s3.list_objects(Bucket = 'udacity-dend')
'''

In [ ]:
'''
import boto3 
s3 = boto3.client("s3")
response = s3.list_objects_v2(
    Bucket='udacity-dend',
    Prefix ='song_data/A',
    MaxKeys=100 )
'''

In [ ]:
# get filepath to song data file
#song_data = "s3://udacity-dend/song_data/"

In [ ]:
#df = spark.read.json("s3a://udacity-dend/song_data/A/B/C/TRABCEI128F424C983.json")

In [ ]:
#print(type(df))
#df

In [ ]:
#df = spark.read.json("s3a://udacity-dend/song_data/")

In [ ]:
#df

In [ ]:
#song_data = "s3a://udacity-dend/song_data/"
#df = spark.read.load(song_data)

In [12]:
#df_songs = df.withColumn("songplay_id", col("song_id"))
#df_songs.show(5)

In [16]:
songs_table = df.select("song_id", "title", "artist_id", "year", "duration", col("artist_name").alias("artist")).distinct() # interesting...when I use .show(), i can't seem to write to a variable, almost as if its
# print statement

In [17]:
print(type(songs_table))

<class 'pyspark.sql.dataframe.DataFrame'>


In [17]:
#songs_table.show(5) # TOO SLOW

+------------------+--------------------+------------------+----+---------+
|           song_id|               title|         artist_id|year| duration|
+------------------+--------------------+------------------+----+---------+
|SOLLALT12A8C1399F3|Piano Concerto No...|ARWMEJW11F4C83C123|   0|319.37261|
|SOAGZUH12A6D4FB4C5|The Sparrows And ...|AR4YEJU1187B991468|1991|191.26812|
|SOSJVZR12AB018698F|Four Simple Steps...|ARJ8IIY1187B992734|2003|  58.8273|
|SORAUDC12A6D4F7273|The Debt Is Settl...|ARJF4E41187FB499FF|1978|233.11628|
|SONXXYP12A81C238EE|Don't Waste My Ti...|ARZNYC61187B9958CF|1985|324.85832|
+------------------+--------------------+------------------+----+---------+
only showing top 5 rows



In [18]:
#df.selectExpr("pos.metadata['song_id'] as songplay_id").limit(5).show(5)

In [19]:
'''
df.createOrReplaceTempView("songtable")
#spark.sql(""" CREATE TABLE IF NOT EXISTS songs (song_id STRING, title STRING, artist_id STRING, year INT, duration float)""")
spark.sql("""
    
    SELECT song_id, title, artist_id, year, duration 
    FROM songtable
""").show()
'''

'\ndf.createOrReplaceTempView("songtable")\n#spark.sql(""" CREATE TABLE IF NOT EXISTS songs (song_id STRING, title STRING, artist_id STRING, year INT, duration float)""")\nspark.sql("""\n    \n    SELECT song_id, title, artist_id, year, duration \n    FROM songtable\n""").show()\n'

In [19]:
songs_table_p = songs_table.write.partitionBy("year", "artist").parquet("songs_table_parquet.parquet")

Py4JJavaError: An error occurred while calling o87.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:226)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:178)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:175)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:213)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:210)
	at

In [ ]:
def process_song_data(spark, input_data, output_data):
    '''
    This function reads the data from S3, processes the song data using Spark, and writes the processed data back to S3.
    '''
    # get filepath to song data file
    song_data = file_locations # might need to add all of the logic here
    
    # read song data file
    df = spark.read.format("json").load(file_locations)

    # extract columns to create songs table
    songs_table = df.select("song_id", "title", "artist_id", "year", "duration").distinct()
    
    # write songs table to parquet files partitioned by year and artist
    songs_table = df.select("song_id", "title", "artist_id", "year", "duration", col("artist_name").alias("artist")).distinct()
    songs_table = songs_table.write.partitionBy("year", "artist").parquet("song_table_parquet.parquet")

    # extract columns to create artists table
    artists_table = df.select("artist_id", "artist_name", "location", "latitidue", "longitude").distinct()
    
    # write artists table to parquet files
    artists_table = artists_table.write.parquet("artists_table_parquet.parquet")

In [ ]:
def process_log_data(spark, input_data, output_data):
   '''
    This function reads the data from S3, processes the log data using Spark, and writes the processed data back to S3.
   '''
    # get filepath to log data file
    log_data = 's3://udacity-dend/log_data/'

    # read log data file
    df = 
    
    # filter by actions for song plays
    df = 

    # extract columns for users table    
    artists_table = 
    
    # write users table to parquet files
    artists_table

    # create timestamp column from original timestamp column
    get_timestamp = udf()
    df = 
    
    # create datetime column from original timestamp column
    get_datetime = udf()
    df = 
    
    # extract columns to create time table
    time_table = 
    
    # write time table to parquet files partitioned by year and month
    time_table

    # read in song data to use for songplays table
    song_df = 

    # extract columns from joined song and log datasets to create songplays table 
    songplays_table = 

    # write songplays table to parquet files partitioned by year and month
    songplays_table

In [ ]:
def main():
    spark = create_spark_session()
    input_data = "s3a://udacity-dend/"
    output_data = ""
    
    process_song_data(spark, input_data, output_data)    
    process_log_data(spark, input_data, output_data)

In [ ]:
if __name__ == "__main__":
    main()

In [ ]:
# SANDBOX AREA BELOW ############

In [ ]:
'''
# This code is from the internet, and it works
from pyspark import SparkContext
sc = SparkContext(master = 'local')
create_spark_session()
'''

In [ ]:
# Works! Stops the spark context
SparkSession.stop()

In [ ]:
song_data = "s3a://udacity-dend/song_data/"
# read song data file
df = spark.read.load(song_data)

In [ ]:
print(song_data)

In [11]:
import boto3
for file in song_data.objects.all():
    print(file)

NameError: name 'song_data' is not defined

In [ ]:
# !pip install boto3
# Also had to install Java SDK

In [ ]:
# Link to udacity bucket with the console:  https://s3.console.aws.amazon.com/s3/buckets/udacity-dend/?region=us-west-2